In [1]:
!pip install tensorflow tensorflow-datasets

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os

In [3]:
# download the dataset
df, info = tfds.load('horses_or_humans',with_info=True, as_supervised = True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1027 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/horses_or_humans/incomplete.RV7VCA_3.0.0/horses_or_humans-train.tfrecord*.…

Generating test examples...:   0%|          | 0/256 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/horses_or_humans/incomplete.RV7VCA_3.0.0/horses_or_humans-test.tfrecord*..…

Dataset horses_or_humans downloaded and prepared to /root/tensorflow_datasets/horses_or_humans/3.0.0. Subsequent calls will reuse this data.


In [4]:
info

tfds.core.DatasetInfo(
    name='horses_or_humans',
    full_name='horses_or_humans/3.0.0',
    description="""
    A large set of images of horses and humans.
    """,
    homepage='http://laurencemoroney.com/horses-or-humans-dataset',
    data_dir=PosixGPath('/tmp/tmp4j2ldtuitfds'),
    file_format=tfrecord,
    download_size=153.59 MiB,
    dataset_size=153.53 MiB,
    features=FeaturesDict({
        'image': Image(shape=(300, 300, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=256, num_shards=1>,
        'train': <SplitInfo num_examples=1027, num_shards=2>,
    },
    citation="""@ONLINE {horses_or_humans,
    author = "Laurence Moroney",
    title = "Horses or Humans Dataset",
    month = "feb",
    year = "2019",
    url = "http://laurencemoroney.com/horses-or-humans-dataset"
    }""",
)

In [5]:
df

{Split('train'): <_PrefetchDataset element_spec=(TensorSpec(shape=(300, 300, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 Split('test'): <_PrefetchDataset element_spec=(TensorSpec(shape=(300, 300, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}

In [6]:
class_names = info.features['label'].names
class_names

['horses', 'humans']

In [7]:
# To save images of training data
for i, example in enumerate(df['train']):
    image,label = example[0],example[1]
    # Define the directory path to save the image
    save_dir = './horse-or-human/train/{}'.format(class_names[label])
    os.makedirs(save_dir, exist_ok=True)
     # Define the filename for the image
    filename = '{}_{}.jpg'.format(class_names[label],i)
    filepath = save_dir + "/" + filename
    # Save the image to the specified file path
    tf.keras.preprocessing.image.save_img(filepath, image)

In [8]:
# To save images of test data
for i, example in enumerate(df['test']):
    image,label = example[0],example[1]
    # Define the directory path to save the image
    save_dir = './horse-or-human/test/{}'.format(class_names[label])
    os.makedirs(save_dir, exist_ok=True)
    # Define the filename for the image
    filename = '{}_{}.jpg'.format(class_names[label],i)
    filepath = save_dir + "/" + filename
    # Save the image to the specified file path
    tf.keras.preprocessing.image.save_img(filepath, image)

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [11]:
# Define the directory where the training images are stored
train_dir = "/content/horse-or-human/train"
image_size = (300,300)
batch_size = 32
# Normalize the images
train_datagen = ImageDataGenerator(rescale=1/255)
# Create a data generator for the training set
train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size = image_size,
                                                   batch_size = batch_size,
                                                   class_mode = 'binary')
# Define the directory where the test images are stored                                                   class_mode= 'binary')
test_dir = "/content/horse-or-human/train"
# Normalize the images
test_datagen = ImageDataGenerator(rescale=1/255)
# Create a data generator for the test set
test_generator = test_datagen.flow_from_directory(test_dir,
                                                   target_size = image_size,
                                                   batch_size = batch_size,
                                                   class_mode= 'binary')


Found 1027 images belonging to 2 classes.
Found 1027 images belonging to 2 classes.


In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam

In [13]:
def get_model():
    model = Sequential()
    # 1st layer CNN
    model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same',activation='relu',input_shape=(300,300,3)))
    model.add(MaxPooling2D(pool_size=2))

    #2nd layer CNN
    model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=2))

    # 3rd layer CNN
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=2))

    # flatten layers and FCN
    model.add(Flatten())
    model.add(Dense(512,activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [14]:
model = get_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 300, 300, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 150, 150, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 150, 150, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 75, 75, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 75, 75, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 37, 37, 128)       0

In [15]:
model.layers

In [16]:
 #Train the model
adam = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator, epochs=5, validation_data = test_generator)


Epoch 1/5
33/33 [==============================] - 306s 9s/step - loss: 0.8333 - accuracy: 0.8325 - val_loss: 0.0610 - val_accuracy: 0.9815
Epoch 2/5
33/33 [==============================] - 305s 9s/step - loss: 0.0399 - accuracy: 0.9903 - val_loss: 0.0239 - val_accuracy: 0.9893
Epoch 3/5
33/33 [==============================] - 305s 9s/step - loss: 0.0534 - accuracy: 0.9883 - val_loss: 0.0365 - val_accuracy: 0.9873
Epoch 4/5
33/33 [==============================] - 308s 9s/step - loss: 0.1877 - accuracy: 0.9611 - val_loss: 0.1218 - val_accuracy: 0.9494
Epoch 5/5
33/33 [==============================] - 324s 10s/step - loss: 0.0971 - accuracy: 0.9688 - val_loss: 0.1845 - val_accuracy: 0.9445


In [17]:
# Save the trained model for future use
model.save('horse-or-human.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
from tensorflow.keras.models import load_model
model_load = load_model('horse-or-human.h5')

In [19]:
train_generator.class_indices

{'horses': 0, 'humans': 1}

In [21]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Use the trained model to predict the class of the image
img = image.load_img("/content/horse-or-human/test/horses/horses_1.jpg",target_size=image_size)
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img/255

prediction = model_load.predict(img)
print(prediction)
if prediction[0][0] >= 0.5:
  print("1 => Human")
else:
  print("0 => Horse")

1/1 [==============================] - 0s 212ms/step
[[4.4510756e-07]]
0 => Horse


In [22]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Use the trained model to predict the class of the image
img = image.load_img("/content/horse-or-human/test/humans/humans_138.jpg",target_size=image_size)
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img/255

prediction = model_load.predict(img)
print(prediction)
if prediction[0][0] >= 0.5:
  print("1 => Human")
else:
  print("0 => Horse")

1/1 [==============================] - 0s 89ms/step
[[1.]]
1 => Human
